In [2]:
# This code takes in the .out files generated from rmsd.tcl and computes the BEST RMSD fits 
# between WNT1/3a/5a and WNT8a. Output is a csv file containing a three column table: 1st column
# contains each WNT1/3a/5a index and the 2nd column contains the WNT8A residue index that best maps 
# to the WNT1/3a/5a residue. The third column is the RMSD value of that fit. Example below:
#
# WNT1/3a/5a INDEX  | WNT8A INDEX | RMSD FIT
#       22          |     55      |    2.9
# .............................................
#
##############
# PURPOSE
##############
# .out files generated contain all possible RMSD fits between WNT1/3a/5a and WNT8a.
# The first column of each row is the index of the residue being fit to on WNT1/3a/5a
# Then, the RMSD fit to each possible residue on WNT8a is output in subsequent columns
#
##############
# HOW RMSD FITTING WORKS
##############
#
# We consider RMSD fits between N residue sequences on WNT1/3a/5a and WNT8.
#
# For Example: We fit residue 60 on WNT1 (residues 40-80) to all possible 20 residue sequences on WNT8a
#
# We take the residue range as follows:
#
# X = Current WNT residue
# N = Sequence Length cutoff.
# A, B = Start/END idx of WNT
# 
#
# IF X - N < A:
#      RANGE = [X, X+2N]
# IF X + N > B:
#      RANGE = [X-2N, X]
# ELSE:
#      RANGE = [X-N, X+N]


import numpy as np
import pandas as pd

# Selector - change SYSTEM variable to generate mappings for each system (1/3a/5a/8a)
SYSTEM = "8a"
if SYSTEM == "1":
    fitwnt = [32,369]
if SYSTEM == "3a":
    fitwnt = [19,352]
if SYSTEM == "5a":
    fitwnt = [44,380]
if SYSTEM == "8a":
    fitwnt = [22,337]


In [9]:
refwnt = [22,337] # Indeces of wnt8a (protein we are mapping to)

offset = 20 # Preselected offset for rmsd fitting (20 = rmsd fitting done over ranges of 20 residues)

align_res100 = np.ones((fitwnt[1] - fitwnt[0] + 1, refwnt[1] - refwnt[0] + offset + 1)) * 500

# Read in the mappings generated by tcl script
with open(f"output/{SYSTEM}_to_8a.out", "r") as file:
    c = 0
    for line in file:
        sp = line.split()
        align_res100[c,0:len(sp)] = sp
        c = c + 1
        

hashmap= []
rmsd_cutoff = 4.5 # RMSD Cutoff, if best rmsd fit is greater than rmsd_cutoff -> mapping idx is set to -1 (no fit)

# For each pairing, check to see what the RMSD of the associated fit was
# Find the minimum RMSD fitting value -> assign idx as best fit/mapping
for i in range(len(align_res100)):
        fit_index = fitwnt[0]+i
        ref_index_at_min_rmsd = np.argmin(align_res100[i,1:])+align_res100[i,0]
        min_rmsd = align_res100[i,1+np.argmin(align_res100[i,1:])]
        if min_rmsd > rmsd_cutoff:
            hashmap.append([fit_index, -1, min_rmsd])
        else:
            hashmap.append([fit_index, ref_index_at_min_rmsd, min_rmsd])
        
# Output the mapping as a CSV file
hashmap = np.array(hashmap)
hashmap_df = pd.DataFrame(hashmap, columns=[SYSTEM,'8a','rmsd'])
hashmap_df[SYSTEM] = hashmap_df[SYSTEM].astype(int)
hashmap_df["8a"] = hashmap_df["8a"].astype(int)
hashmap_df.to_csv(f"output/{SYSTEM}_to_8a_map.csv", index=False)
hashmap_df

(316, 336)
348
22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287

,8a,8a,rmsd
0,22,22,0.000000
1,23,23,0.000002
2,24,24,0.000002
3,25,25,0.000003
4,26,26,0.000003
...,...,...,...
311,333,333,0.000019
312,334,334,0.000020
313,335,335,0.000021
314,336,336,0.000022


In [3]:
import matplotlib.pyplot as plt
plt.plot(hashmap_df[SYSTEM], hashmap_df["rmsd"])
plt.xlabel("Wnt3A Residue ID", fontsize=20)
plt.ylabel(f"Minimum RMSD ($\AA$)", fontsize=20)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

plt.savefig(f"{SYSTEM}_to_8a_map.png", dpi=400)

<>:4: SyntaxWarning: invalid escape sequence '\A'
<>:4: SyntaxWarning: invalid escape sequence '\A'
/var/folders/p_/mmkyzkc1655_rf8nypyzqx7r0000gn/T/ipykernel_12011/3819731559.py:4: SyntaxWarning: invalid escape sequence '\A'
  plt.ylabel(f"Minimum RMSD ($\AA$)", fontsize=20)
/var/folders/p_/mmkyzkc1655_rf8nypyzqx7r0000gn/T/ipykernel_12011/3819731559.py:4: SyntaxWarning: invalid escape sequence '\A'
  plt.ylabel(f"Minimum RMSD ($\AA$)", fontsize=20)


ModuleNotFoundError: No module named 'matplotlib'